In [1]:
import matplotlib
matplotlib.use('PDF')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import pickle
plt.style.use('grayscale')

In [3]:
def human_format(x):
    sigpart = 0
    suffix = ""
    if x < 1000:
        return "{:.3g}".format(x)
    elif x < 1000000:
        sigpart = x / 1000
        suffix = "k"
    elif x < 1000000000:
        sigpart = x / 1000000
        suffix = "M"
    else:
        sigpart = x / 1000000000
        suffix = "B"
    return "{:.3g} {}".format(sigpart, suffix)

# P

In [27]:
pacc = pd.read_csv("./usecase_p.csv")
pacc.head(1)

,algorithm,params,Agree,Predicted,Actual,Precision,Recall,F1
0,kde,bw=0.27,143,161,193,0.888199,0.740933,0.80791


In [28]:
algorithms = ["histogram", "kde", "knn"]
colors = ["0.2", "0.7", "1.0"]

In [44]:
fig, axs = plt.subplots(nrows=1, ncols=1, dpi=300, figsize=(6,3))
ax = axs
labels = []
for ai, alg in enumerate(algorithms):
    data = pacc[pacc.algorithm == alg]
    labels += list(data.params)
    xlocs = (np.arange(0,3) + ai*4)
    ax.bar(left=xlocs,height=data.F1, label=alg, color=colors[ai])
ax.set_xticks(np.array([0,1,2,4,5,6,8,9,10]))
ax.set_xlim(np.array([-0.25,3])*4)
ax.set_xticklabels(labels, rotation=-45, ha='left', va='top', x=-50)
# ax.set_xticklabels(labels, rotation="vertical")
ax.yaxis.grid(True)
ax.set_yticks(np.arange(0,1.1,0.25))
ax.set_xlabel("Algorithm")
ax.set_ylabel("F-Score")
ax.set_title("Probability Density Accuracy")
# box = ax.get_position()
# ax.set_position([box.x0, box.y0 + box.height * 0.1,
#                  box.width, box.height * 0.9])

ax.legend(loc='lower center', 
          bbox_to_anchor=(0.5, -1.5),
         ncol=3)
fig.tight_layout(rect=[0, .15, 1, 1])
fig.savefig("../figures/pdfacc.pdf")
fig.clear()

/usr/local/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/local/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


# Region

In [47]:
racc = pd.read_csv("./usecase_region.csv")
racc.head(1)

,algorithm,params,Agree,Predicted,Actual,Precision,Recall,F1
0,kde,bw=0.27,5123,5172,5283,0.990526,0.969714,0.98001


In [46]:
algorithms = ["histogram", "kde", "knn"]
colors = ["0.2", "0.7", "1.0"]

In [55]:
fig, axs = plt.subplots(nrows=1, ncols=1, dpi=300, figsize=(6,3))
ax = axs
labels = []
for ai, alg in enumerate(algorithms):
    data = racc[racc.algorithm == alg]
    labels += list(data.params)
    xlocs = (np.arange(0,3) + ai*4)
    ax.bar(left=xlocs,height=data.F1, label=alg, color=colors[ai])
ax.set_xticks(np.array([0,1,2,4,5,6,8,9,10]))
ax.set_xlim(np.array([-0.25,3])*4)
ax.set_xticklabels(labels, rotation=-45, ha='left', va='top', x=-50)
# ax.set_xticklabels(labels, rotation="vertical")
ax.yaxis.grid(True)
ax.set_yticks(np.arange(0,1.1,0.25))
ax.set_xlabel("Algorithm")
ax.set_ylabel("F-Score")
ax.set_title("Region Contour Accuracy")
# box = ax.get_position()
# ax.set_position([box.x0, box.y0 + box.height * 0.1,
#                  box.width, box.height * 0.9])

ax.legend(loc='lower center', 
          bbox_to_anchor=(0.5, -1.5),
         ncol=3)
fig.tight_layout(rect=[0, .15, 1, 1])
fig.savefig("../figures/regionacc.pdf")
fig.clear()

/usr/local/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/local/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")


# Outlier

In [65]:
oacc = pd.read_csv("./outlieracc_shuttle.csv")
oacc.head(1)

,algorithm,Parameters,TruePos,Pos,Anomalies,Runtime
0,MCD,NaN,0,286,186,0.018


In [66]:
oacc["precision"] = oacc.TruePos/oacc.Pos
oacc["recall"] = oacc.TruePos/oacc.Anomalies
oacc["F1"] = oacc["precision"] * oacc["recall"] * 2 / (oacc.precision + oacc.recall)

In [81]:
oacc["F1"] = oacc.F1.fillna(0)

In [89]:
def make_factor_chart(ax, labels, ys):
    xs = np.array(range(len(ys)))
    ax.bar(
        xs + 0.5,
        ys,
    )
    rects = ax.patches
    for rect, label in zip(rects, ys):
        height = rect.get_height()
        ax.text(
            rect.get_x() + rect.get_width()/2, 
            height,
            human_format(float(label)),
            ha='center', va='bottom', size = 8,
        )
    ax.set_xticks(xs + 0.5)
    ax.set_xticklabels(labels, rotation=-25, ha='left', va='top',)
    ax.yaxis.grid(True)
    return ax

In [91]:
fig, axs = plt.subplots(ncols=1, figsize=(6,3),dpi=300)
ax = axs
make_factor_chart(ax, oacc.algorithm, oacc.F1)
ax.set_ylim(0,1.0)
ax.set_yticks(np.arange(0,1.1,0.25))
ax.set_title("Detecting Rare Shuttle States")
ax.set_ylabel("F1 Score")
ax.set_xlabel("Algorithm")
fig.tight_layout()
fig.savefig("../figures/outliers_2.pdf")
fig.clear()

/usr/local/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/local/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
